## First, imports:

In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units

Import my library:

In [ ]:
import os
import sys

apt_path = os.path.abspath(os.path.join('..', 'apostletools'))
sys.path.append(apt_path)

import snapshot
import dataset_comp
import curve_fit

In [ ]:
import importlib
importlib.reload(snapshot)
importlib.reload(dataset_comp)
importlib.reload(curve_fit)

# Circular Velocities Near Half-light Radius

Here, I plot the circular velocities at radius of 1 kpc, against the galaxy masses ($v_\mathrm{max}$).

## Motivation

1 kpc is of the order of what would expectedly be the half-light radii of these dwarf galaxies. I expect a systematic reduction in this relation in the spectator model. This expectation is validated by the differences in the density profiles (or rotation curves) between the models. 

---

## Set Parameters for the Plots

Choose the snapshot and the simulations, and define M31 and MW in each simulation. Also, set the colors used for each simulation:

In [ ]:
snap_id = 127
sim_ids = ['V1_MR_fix', 'V1_MR_curvaton_p082_fix']
names = ['plain-LCDM', 'curv-p082']
colors = [['black', 'gray'], ['red', 'pink']]

m31 = [(1,0), (1,0)]
mw = [(2,0), (1,1)]

Choose how to distinguish between satellite and isolated galaxies:

In [ ]:
distinction = 'by_r'
maxdi = 2000 # Maximum distance from LG centre for isolated

Set lower limit for the value of $v_\mathrm{max}$ of the galaxies to be included (separately for satellites and isolated galaxies):

In [ ]:
sat_low = 10
isol_low = 10

---

## Retrieve Data

### Create a Dictionary

For easy handling of the relevant data, define a data dictionary that, at the top level, has entries for all simulations. Under each simulation entry, add items for the needed datasets and, under the 'Selections' key, a sub-dictionary of masking arrays for each needed condition (e.g. satellite, luminous, $v_\mathrm{max}$ inside range, etc.).

First, add the above definitions into the data dict:

In [ ]:
data = {}
for name, sim_id, m31_ns, mw_ns, col in \
    zip(names, sim_ids, m31, mw, colors):
    data[name] = {'snapshot': snapshot.Snapshot(sim_id, snap_id, name=name),
                  'M31_identifier': m31_ns,
                  'MW_identifier': mw_ns,
                  'PlotStyle': {
                      'Color': col
                  }
                 }

Then, loop over simulations, retrieve data, compute masking arrays, and add to the dictionary:

In [ ]:
for name, sim_data in data.items():
    # Get data:
    snap = sim_data["snapshot"]
    max_point = snap.get_subhalos("Max_Vcirc", "Extended")
    vmax = max_point[:,0] * units.cm.to(units.km)
    v1kpc = snap.get_subhalos("V1kpc", "Extended") * units.cm.to(units.km)
    
    # Split into satellites:
    mask_m31, mask_mw, mask_isol = dataset_comp.split_satellites_by_distance(
        snap, sim_data["M31_identifier"], sim_data["MW_identifier"]
    )
        
    mask_lum, mask_dark = dataset_comp.split_luminous(snap)
    
    # Prune low-mass:
    mask_sat_low = dataset_comp.prune_vmax(snap, low_lim=sat_low)
    mask_isol_low = dataset_comp.prune_vmax(snap, low_lim=isol_low)
    mask_m31 = np.logical_and(mask_m31, mask_sat_low)
    mask_mw = np.logical_and(mask_mw, mask_sat_low)
    mask_sat = np.logical_and(np.logical_or(mask_m31, mask_mw), mask_sat_low)
    mask_isol = np.logical_and(mask_isol, mask_isol_low)

    # Add selections (masking arrays):
    data[name]['Selections'] = {
        'M31': mask_m31,
        'MW': mask_mw,
        'Satellite': mask_sat,
        'Isolated': mask_isol,
        'Luminous': mask_lum,
        'Dark': mask_dark
    }
        
    # Add datasets to dictionary:
    data[name]['Vmax'] = vmax
    data[name]['V1kpc'] = v1kpc

---

## Plot

Plot satellites and isolated galaxies on different subplots and add median fits for each dataset.

First, set figure parameters:

In [ ]:
# Choose font sizes:
parameters = {'axes.titlesize': 12,
              'axes.labelsize': 10,
              'xtick.labelsize': 9,
              'ytick.labelsize': 9,
              'legend.fontsize': 10}

ms = 10 # Marker size
msl = 15 # " " for luminous
a = 0.9 # Transparency

# Set fonts:
plt.rcParams.update(parameters)
plt.tight_layout()

In [ ]:
fig, axes = plt.subplots(ncols=2, sharey='row', figsize=(6, 3))
plt.subplots_adjust(wspace=0.05)

# Set axis:
for ax in axes:
    ax.set_xscale('log')
axes[0].set_xlim(10, 110)    
axes[1].set_xlim(10, 110)
axes[0].set_xlabel('$v_{\mathrm{max}}[\mathrm{km s^{-1}}]$')
axes[1].set_xlabel('$v_{\mathrm{max}}[\mathrm{km s^{-1}}]$')

axes[0].set_yscale('log')
axes[0].set_ylim(5, 70)    
axes[0].set_ylabel('$v_{\mathrm{1 kpc}} [\mathrm{kms^{-1}}]$')

axes[0].set_title('Satellite Subhalos' )
axes[1].set_title('Isolated Subhalos' )

# Add scatter plots:
for i, (name, entry) in enumerate(data.items()):
    
    # Plot dark:
    mask = np.logical_and(entry['Selections']['Satellite'],
                          entry['Selections']['Dark'])
    axes[0].scatter(entry['Vmax'][mask], entry['V1kpc'][mask], 
                    s=ms, edgecolor='none', alpha=a,
                    c=entry['PlotStyle']['Color'][1])
    
    mask = np.logical_and(entry['Selections']['Isolated'],
                          entry['Selections']['Dark'])
    axes[1].scatter(entry['Vmax'][mask], entry['V1kpc'][mask], 
                    s=ms, edgecolor='none', alpha=a,
                    c=entry['PlotStyle']['Color'][1])
    
    # Plot luminous:
    mask = np.logical_and(entry['Selections']['Satellite'],
                          entry['Selections']['Luminous'])
    axes[0].scatter(entry['Vmax'][mask], entry['V1kpc'][mask], 
                    s=msl, facecolors='none', alpha=a,
                    edgecolors=entry['PlotStyle']['Color'][1])
    
    mask = np.logical_and(entry['Selections']['Isolated'],
                          entry['Selections']['Luminous'])
    axes[1].scatter(entry['Vmax'][mask], entry['V1kpc'][mask], 
                    s=msl, facecolors='none', alpha=a,
                    edgecolors=entry['PlotStyle']['Color'][1])

In [ ]:
for sim_name, sim_data in data.items():
    axes[0].scatter([], [], c=sim_data['PlotStyle']['Color'][1], alpha=a,
                s=ms, label=sim_name)
    
axes[0].legend(loc='upper left')

axes[1].scatter([], [], edgecolor='black', facecolor='none', alpha=a,
                s=msl, label="Luminous")
axes[1].scatter([], [], c='black', alpha=a, s=ms, label="Dark")
axes[1].legend(loc='lower right')

fig

### Add Median Curves

In [ ]:
n_median_bins = 5
for i, (name, entry) in enumerate(data.items()):
    
    # For satellites:
    mask = entry['Selections']['Satellite']
    x = np.log10(entry['Vmax'][mask])
    y = np.log10(entry['V1kpc'][mask])
    median = curve_fit.median_trend_fixed_bin_width(
        x, y, n_bins=n_median_bins
    )
    if median is not None:
        axes[0].plot(10**median[0], 10**median[1], 
                     entry['PlotStyle']['Color'][0], linestyle='--')
    else:
        print("Could not fit median for:", name)
    
    # For isolated:
    mask = entry['Selections']['Isolated']
    x = np.log10(entry['Vmax'][mask])
    y = np.log10(entry['V1kpc'][mask])
    median = curve_fit.median_trend_fixed_bin_width(
        x, y, n_bins=n_median_bins
    )
    if median is not None:
        axes[1].plot(10**median[0], 10**median[1], 
                     entry['PlotStyle']['Color'][0], linestyle='--')
    else:
        print("Could not fit median for:", name)
        
fig

### Save the Figures

In [ ]:
filename = 'v1kpc_vs_vmax.png'
    
path = os.path.abspath(os.path.join('..', 'Figures', 'MediumResolution'))
filename = os.path.join(path, filename)

fig.savefig(filename, dpi=300, bbox_inches='tight')